# Safety in Construction Content Summarizer
Simple LLM project using OpenAI API to summarize content of a website and give actionable recommendations based on safety concern discussed in the content.

In [5]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key
if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [3]:
openai = OpenAI()

In [4]:
# Scrape from website using simple scraper
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class ScrapeWebsite:
    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [8]:
class SummarizeContent:
    def __init__(self, web_url):
        self.website = ScrapeWebsite(web_url)
        self.title = self.website.title
        self.text = self.website.text

    def prompt(self):
        system_prompt = "You are a construction safety expert assistant that analyzes the contents of a website \
            and provides a short summary about the hazards explained in the website, ignoring text that might be navigation related. \
            Respond in markdown."

        user_prompt = f"You are looking at a website titled {self.website.title}"
        user_prompt += "\nThe contents of this website is as follows; \
            please provide a short summary of this website in markdown. \
            Provides recommendation action for hazards in the content.\n\n"
        user_prompt += self.website.text
        
        return [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]

    def summarize(self):
        stream = openai.chat.completions.create(
            model='gpt-4o-mini', 
            messages=self.prompt(), 
            stream=True)

        response = ""
        display_handle = display(Markdown(""), display_id=True)
        for chunk in stream:
            response += chunk.choices[0].delta.content or ''
            response = response.replace("```","").replace("markdown", "")
            update_display(Markdown(response), display_id=display_handle.display_id)

In [9]:
# Test on public website
summarizer = SummarizeContent("https://www.rakenapp.com/features/toolbox-talks/3-poor-housekeeping-hazards")
summarizer.summarize()

# Summary of Poor Housekeeping Hazards Toolbox Talk - Raken

The **Poor Housekeeping Hazards Toolbox Talk** emphasizes the importance of maintaining a clean and organized workplace to ensure safety in construction settings. It identifies three main types of hazards stemming from poor housekeeping:

1. **Strain and Sprain Hazards**
   - Hazards arise from the improper placement of tools and materials, leading to potential trips and falls.
   - **Recommendation:** Ensure items are stored correctly off the floor to minimize risks. Return tools to their designated places after use.

2. **Laceration Hazards**
   - Sharp objects and tools can cause cuts or lacerations if not properly managed.
   - **Recommendation:** Clean up sharp items immediately and store them securely post-use to prevent accidental injuries.

3. **Slip, Trip, and Fall Hazards**
   - Spills and clutter can significantly increase the risk of slips, trips, and falls.
   - **Recommendation:** Promptly address spills and ensure that walkways are clear and well-marked, especially in areas with changes in flooring levels.

## Actionable Steps for Mitigating Hazards
- Keep all floors dry and organized to prevent slipping or tripping.
- Regularly inspect stairways and hallways, keeping them clear of clutter and cleaned.
- Properly label any areas containing toxic substances.
- Ensure safety equipment (fire extinguishers, emergency exits) is easily accessible and correctly placed.
- Maintain cleanliness in restrooms and ensure waste disposal is frequent and organized.

In conclusion, fostering a culture of good housekeeping practices is crucial for improving workplace safety and employee productivity. Each worker is encouraged to take responsibility for their workspace and uphold safety standards for the benefit of everyone on the job site.